In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform
from scipy.stats import norm
import japanize_matplotlib

# グラフのフォント設定
plt.rcParams['font.size'] = 12

# 入出力ファイルの設定
INPUT_FILE_NAME = 'data.csv'  
OUTPUT_IMAGE_NAME = 'figure_11_polychoric_heatmap.png'
OUTPUT_CSV_NAME = 'correlation_matrix.csv'
ENCODING_TYPE = 'shift_jis' 

def calculate_polychoric_correlation(x, y):
    """
    ポリコリック相関の近似計算（Greinerの式）
    """
    # 欠損値除去
    data = pd.DataFrame({'x': x, 'y': y}).dropna()
    x_clean = data['x'].values
    y_clean = data['y'].values
    
    if len(x_clean) == 0:
        return np.nan

    # スピアマン順位相関係数の算出
    r_spearman = pd.Series(x_clean).corr(pd.Series(y_clean), method='spearman')
    
    # 近似式による変換: r_poly = 2 * sin(π * r_spearman / 6)
    r_polychoric_approx = 2 * np.sin(np.pi * r_spearman / 6)
    
    return r_polychoric_approx

# データの読み込み
try:
    df = pd.read_csv(INPUT_FILE_NAME, encoding=ENCODING_TYPE)
except UnicodeDecodeError:
    df = pd.read_csv(INPUT_FILE_NAME, encoding='utf-8')

# 分析対象カラムの定義とリネーム
column_mapping = {
    '年齢を教えてください。': '年齢',
    '甘味をどの程度感じましたか？（１／５）': '甘味',
    'うま味をどの程度感じましたか？（２／５）': 'うま味',
    '苦味をどの程度感じましたか？（３／５）': '苦味',
    '酸味をどの程度感じましたか？（４／５）': '酸味',
    '塩味をどの程度感じましたか？（５／５）': '塩味',
    '華やかさをどの程度感じましたか？（１／５）': '華やかさ',
    '開放感をどの程度感じましたか？（２／５）': '開放感',
    '爽やかさをどの程度感じましたか？（３／５）': '爽やかさ',
    '賑い感をどの程度感じましたか？（４／５）': '賑わい感',
    '賑わい感をどの程度感じましたか？（４／５）': '賑わい感',
    '高揚感をどの程度感じましたか？（５／５）': '高揚感',
    'クラフトビール「白蒲田」をまた飲んでみたいと思いますか？': 'リピート意向'
}

# 必要な列の抽出
valid_cols = [col for col in column_mapping.keys() if col in df.columns]
df_analysis = df[valid_cols].rename(columns=column_mapping)
df_analysis = df_analysis.loc[:, ~df_analysis.columns.duplicated()]

# 数値型への変換
for col in df_analysis.columns:
    df_analysis[col] = pd.to_numeric(df_analysis[col], errors='coerce')

# 相関行列の算出
print("ポリコリック相関（近似）を計算中...")

cols = df_analysis.columns
n_cols = len(cols)
correlation_matrix = pd.DataFrame(np.zeros((n_cols, n_cols)), index=cols, columns=cols)

for i in range(n_cols):
    for j in range(n_cols):
        if i == j:
            correlation_matrix.iloc[i, j] = 1.0
        else:
            rho = calculate_polychoric_correlation(df_analysis.iloc[:, i], df_analysis.iloc[:, j])
            correlation_matrix.iloc[i, j] = rho

# 行列の対称化処理（計算誤差の補正）
correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2
np.fill_diagonal(correlation_matrix.values, 1.0)

# 結果の確認と保存
print("算出された相関行列:")
display(correlation_matrix.round(3))
correlation_matrix.to_csv(OUTPUT_CSV_NAME)
print(f"相関行列を保存しました: {OUTPUT_CSV_NAME}")

# 距離行列の作成
distance_matrix = 1 - correlation_matrix.abs()
# 距離行列も対称化
distance_matrix = (distance_matrix + distance_matrix.T) / 2
np.fill_diagonal(distance_matrix.values, 0)

# 階層型クラスタリング（Ward法）
linkage_matrix = linkage(squareform(distance_matrix.values), method='ward')

# ヒートマップの描画
plt.figure(figsize=(12, 10))

g = sns.clustermap(
    correlation_matrix,
    row_linkage=linkage_matrix,
    col_linkage=linkage_matrix,
    row_cluster=True,
    col_cluster=True,
    cmap="coolwarm",
    annot=True,
    fmt=".2f",
    figsize=(12, 10),
    linewidths=0.5,
    dendrogram_ratio=(0.15, 0.15),
    cbar_pos=(0.02, 0.8, 0.03, 0.18),
    vmax=1.0,
    vmin=-1.0,
    center=0
)

g.fig.suptitle("変数間の相関構造 (ポリコリック相関 x Ward法)", fontsize=16, y=1.02)

plt.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
plt.show()